Source practice: https://www.youtube.com/watch?v=Jy4wM2X21u0&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=3

##Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

##Load data

In [5]:
train_dataset = datasets.MNIST(download = True, root = 'dataset/', train = True, transform = transforms.ToTensor())

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 54387930.40it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1946893.18it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 13802245.99it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7798006.05it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [6]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

In [7]:
test_dataset = datasets.MNIST(download = True, root = 'dataset/', train = False, transform = transforms.ToTensor())

In [8]:
test_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

##Set device

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyperparameters

In [17]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

# RNN model

In [18]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    #Forward
    out, _ = self.rnn(x, h0)
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out

##Initialize network

In [19]:
model = RNN(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, num_classes = num_classes).to(device)

##Loss and Optimizer

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Train Network

In [21]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    #Get data to cuda if possible
    data = data.to(device = device).squeeze(1) ####
    targets = targets.to(device = device)

    #Forward
    scores = model(data)
    loss = criterion(scores, targets)

    #Backward
    optimizer.zero_grad()
    loss.backward()

    #Gradient descent or adam step
    optimizer.step()


## Check accuracy on training and test

In [22]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device).squeeze(1)
      y = y.to(device = device)

      scores = model(x)
      _, predictions = scores.max(1) #maximum values
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()
  return num_correct

In [23]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58073 / 60000 with accuracy 96.79
Checking accuracy on training data
Got 58073 / 60000 with accuracy 96.79


tensor(58073)